In [36]:
import os
import numpy as np
import cupy as cp
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import tqdm
import json


In [37]:
def random_crop(img, crop_size=(256, 256)):
    """
    Randomly crops an image to the specified size.

    Parameters:
        img (numpy.ndarray): The input image to be cropped.
        crop_size (tuple): The size of the crop (height, width). Default is (256, 256).

    Returns:
        numpy.ndarray: The cropped image.

    Raises:
        ValueError: If the input image is smaller than the specified crop size.
    """
    img_height, img_width = img.shape[:2]

    if img_height < crop_size[0] or img_width < crop_size[1]:
        raise ValueError("Image is too small for the desired crop size.")

    y = np.random.randint(0, img_height - crop_size[0] + 1)
    x = np.random.randint(0, img_width - crop_size[1] + 1)

    return img[y:y + crop_size[0], x:x + crop_size[1]]

def image_normalization(img):
    """
    Normalize the input image.

    This function normalizes the input image by scaling its pixel values to the range [0, 1] 
    if the maximum pixel value is greater than 1.

    Parameters:
        img (numpy.ndarray): The input image to be normalized.

    Returns:
        numpy.ndarray: The normalized image with pixel values in the range [0, 1].
    """
    if img.max() > 1:
        img = img / 255.0
    return img

def texture_masking(foreground_texture, background_texture, mask):
    """
    Apply texture masking to blend a foreground texture and a background texture using a mask.
    
    Parameters:
        foreground_texture (numpy.ndarray): The texture to be applied in the foreground.
        background_texture (numpy.ndarray): The texture to be applied in the background.
        mask (numpy.ndarray): The mask used to blend the foreground and background textures. 
                          Values should be normalized between 0 and 1.
    Returns:
        numpy.ndarray: The resulting image after applying the texture masking.
    """
    # Normalize inputs
    foreground_texture = image_normalization(foreground_texture)
    background_texture = image_normalization(background_texture)
    mask = image_normalization(mask)
    
    textured_img = mask * foreground_texture + (1 - mask) * background_texture
    
    return textured_img

def validate_image_dimensions(img, target_channels=3):
    """
    Ensures the image has the correct number of channels.

    Parameters:
        img (ndarray): The input image array. It can be a grayscale image (2D array) or a color image (3D array).
        target_channels (int, optional): The desired number of channels for the output image. Default is 3.

    Returns:
        ndarray: The image array with the correct number of channels.

    Raises:
        ValueError: If the input image has an unexpected number of channels.
    """
    """Ensures the image has the correct number of channels."""
    if len(img.shape) == 2:  # Grayscale image
        img = cp.expand_dims(img, axis=-1)  # (H, W) -> (H, W, 1)
        img = cp.repeat(img, target_channels, axis=-1)  # (H, W, 1) -> (H, W, 3)
    elif img.shape[2] != target_channels:  # Unexpected number of channels
        raise ValueError(f"Image has {img.shape[2]} channels, but {target_channels} channels are required.")
    return img

def validate_image_size(foreground_texture, background_texture, crop_size):
    """
    Validates if the given foreground and background textures are large enough to be cropped to the specified size.
    Parameters:
        foreground_texture (numpy.ndarray): The foreground texture image as a NumPy array.
        background_texture (numpy.ndarray): The background texture image as a NumPy array.
        crop_size (tuple): A tuple (height, width) specifying the desired crop size.
    Returns:
        bool: True if both textures are large enough to be cropped to the specified size, False otherwise.
    """
    
    if (foreground_texture.shape[0] >= crop_size[0] and foreground_texture.shape[1] >= crop_size[1] and
        background_texture.shape[0] >= crop_size[0] and background_texture.shape[1] >= crop_size[1]):        
        return True
    else:
        return False
        

def texture_transfer(mask_dir, texture_dir, output_dir, img_labels=None, crop_size=(256, 256)):
    """
    Transfers textures onto masks and saves the resulting images.
    
    Parameters:
        mask_dir (str): Directory containing mask images. Mask images should be in '.png', '.tif', or '.tiff' format.
        texture_dir (str): Directory containing texture images. Texture images should be in '.png', '.jpg', or '.JPEG' format.
        output_dir (str): Directory where the output images will be saved.
        crop_size (tuple): Size of the crop to be applied to the textures. Default is (256, 256).
    Returns:
    None
    """
    # Get list of files in the directories
    mask_files = [os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith(('.png', '.tif', '.tiff'))]
    texture_files = [os.path.join(texture_dir, f) for f in os.listdir(texture_dir) if f.endswith(('.png', '.jpg', '.JPEG'))]
    

    labels_df = pd.read_csv(img_labels)
    
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    all_metadata = []
       
    for mask_path in tqdm.tqdm(mask_files):

        
        # Load the mask
        mask = Image.open(mask_path).convert('L')  # Convert to grayscale
        mask = cp.asarray(mask) / 255.0  # Normalize mask to range [0, 1]
        mask_name = os.path.basename(mask_path).split('.')[0]
        
        # Randomly pick 2 textures
        while True:
            
            foreground_texture_path, background_texture_path = np.random.choice(texture_files, 2, replace=False)
            
            # Load the textures
            foreground_texture = Image.open(foreground_texture_path).convert('RGB')
            background_texture = Image.open(background_texture_path).convert('RGB')
            
            foreground_texture = cp.asarray(foreground_texture)
            background_texture = cp.asarray(background_texture)
            
            foreground_texture_name = os.path.basename(foreground_texture_path).split('.')[0]
            background_texture_name = os.path.basename(background_texture_path).split('.')[0]
            
            foreground_texture = validate_image_dimensions(foreground_texture)
            background_texture = validate_image_dimensions(background_texture)            
            
            foreground_class = labels_df.loc[labels_df['image_id'] == foreground_texture_name, 'label'].values[0]
            background_class = labels_df.loc[labels_df['image_id'] == background_texture_name, 'label'].values[0]
            
            # check if textures are from different classes and large enough for cropping
            if foreground_class != background_class and validate_image_size(foreground_texture, background_texture, crop_size):
                break #exit the loop
                
        # Crop the textures
        foreground_texture_crop = random_crop(foreground_texture, crop_size)
        background_texture_crop = random_crop(background_texture, crop_size)
        
        mask = validate_image_dimensions(mask)
        
        # Apply texture masking
        textured_img = texture_masking(foreground_texture_crop, background_texture_crop, mask)
        
        # Convert cupy array back to numpy array before saving
        textured_img = cp.asnumpy(textured_img)
        
        images_metadata = {
            'file_name': f'{mask_name}.png',
            'foreground_texture_name': foreground_texture_name,
            'background_texture_name': background_texture_name,
            'foreground_class': foreground_class,
            'background_class': background_class
        }
        
        all_metadata.append(images_metadata)
        
        # Save the output
        output_path = os.path.join(output_dir, f'{mask_name}.png')
        Image.fromarray((textured_img * 255).astype('uint8')).save(output_path)
        
        with open(os.path.join(output_dir, 'metadata.json'), 'w') as f:
            json.dump(all_metadata, f, indent=4)
        
        



In [38]:
# Define directories
mask_dir = '/media/wesleygalvao/1_TB_LINUX/Datasets/VessShape/curves/mask'
texture_dir = '/media/wesleygalvao/1_TB_LINUX/Datasets/ImageNet/ILSVRC2012_img_val'
output_dir = '/media/wesleygalvao/1_TB_LINUX/Datasets/VessShape/curves/images'
img_labels = '/media/wesleygalvao/1_TB_LINUX/Datasets/ImageNet/ILSVRC2012_img_val_annotation.csv'

# Perform texture transfer
texture_transfer(mask_dir, texture_dir, output_dir, img_labels=img_labels, crop_size=(256, 256))

100%|██████████| 70000/70000 [3:11:17<00:00,  6.10it/s]  


In [9]:
img_labels_df = pd.read_csv(img_labels)

In [10]:
img_labels_df

,image_id,label_id,label
0,ILSVRC2012_val_00048981,n03995372,power drill
1,ILSVRC2012_val_00037956,n03481172,hammer
2,ILSVRC2012_val_00026161,n02108000,EntleBucher
3,ILSVRC2012_val_00026171,n03109150,"corkscrew, bottle screw"
4,ILSVRC2012_val_00008726,n02119789,"kit fox, Vulpes macrotis"
...,...,...,...
49995,ILSVRC2012_val_00005961,n03388043,fountain
49996,ILSVRC2012_val_00008801,n03089624,"confectionery, confectionary, candy store"
49997,ILSVRC2012_val_00008176,n01518878,"ostrich, Struthio camelus"
49998,ILSVRC2012_val_00004764,n03874293,"paddlewheel, paddle wheel"


In [21]:
img_labels_df.loc[img_labels_df['image_id'] == 'ILSVRC2012_val_00014725', 'label'].values[0]

'padlock'